# Importing Dependincies

In [89]:
# !pip install nbformat --upgrade

In [90]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors
import datetime as dt

In [91]:
# Loading dataset.
retails = pd.read_csv("online_retail.csv")

df = retails.copy()

df.head(5) # Show first 5 row.

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [92]:
df.shape # How many element? And features in the dataset.

(541909, 8)

In [93]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [94]:
df.isna().sum() # Check NaN values.

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [95]:
df.dropna(subset="CustomerID", inplace=True) # Drop row if CustomerID is NaN

In [96]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"],) # Convert datetime.
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [97]:
df["TotalAmount"] = df["Quantity"] * df["UnitPrice"]

In [98]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [99]:
reference_date = pd.Timestamp(dt.datetime.now().date()) # Getting reference date for 14 years ago.

In [100]:
reference_date = df["InvoiceDate"].max() + dt.timedelta(days=1)

In [101]:
rfm = df.groupby("CustomerID").agg({
    "InvoiceDate": lambda x: (reference_date - x.max()).days,
    "InvoiceNo": "count",
    "TotalAmount": "sum"
})

In [102]:
rfm.rename(columns={"InvoiceDate": "Recency", "InvoiceNo": "Frequency", "TotalAmount": "Monetary"}, inplace=True) # Statistics for each customer.

In [103]:
rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [104]:
quantiles = rfm.quantile(q=[.25, .5, .75])

# Assign RFM Score
def RScore(x, p, d):

    # X -> Value
    # p -> Column Name

    if  p == "Recency": # Most recent customers get higher RScore.
        if x <= d[p][.25]:
            return 4
        elif x <= d[p][.5]:
            return 3
        elif x <= d[p][.75]:
            return 2
        else:
            return 1
    else: # Most purchaser get higher RScore.
        if x <= d[p][.25]:
            return 1
        elif x <= d[p][.5]:
            return 2
        elif x <= d[p][.75]:
            return 3
        else:
            return 4
        


rfm["R"] = rfm["Recency"].apply(RScore, args = ("Recency", quantiles,))
rfm["F"] = rfm["Frequency"].apply(RScore, args = ("Frequency", quantiles,))
rfm["M"] = rfm["Monetary"].apply(RScore, args = ("Monetary", quantiles,))

In [105]:
rfm.head()

,Recency,Frequency,Monetary,R,F,M
CustomerID,,,,,,
12346.0,326,2,0.00,1,1,1
12347.0,2,182,4310.00,4,4,4
12348.0,75,31,1797.24,2,2,4
12349.0,19,73,1757.55,3,3,4
12350.0,310,17,334.40,1,1,2


In [106]:
rfm['RFM_Segment'] = rfm["R"].astype(str) + rfm["F"].astype(str) + rfm["M"].astype(str)
rfm["RFM_Score"] = rfm[["R", "F", "M"]].sum(axis=1)

In [107]:
rfm.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3
12347.0,2,182,4310.00,4,4,4,444,12
12348.0,75,31,1797.24,2,2,4,224,8
12349.0,19,73,1757.55,3,3,4,334,10
12350.0,310,17,334.40,1,1,2,112,4


RFM_Segment
444    471
111    392
122    209
344    206
211    181
      ... 
124      6
142      4
414      3
241      1
441      1
Name: count, Length: 62, dtype: int64